In [1]:
import cv2
from cv2 import aruco
import numpy as np
from matplotlib import pyplot as plt
import time

#fonction Sophie
from findPath import findPath

#fonction Liandro
from camera import read_camera_image
#from get_corner import get_corner
#from get_obstacles_coordinates import get_obstacles_coordinates

In [2]:
def get_corner(gray):
    """
    Get the coordinates of the corners on the real camera image
    """
    height, width = gray.shape
    
    # Initialization of the ArUco dictionary
    aruco_dict = aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
    parameters = aruco.DetectorParameters()
    
    keypoints_coordinates = []
    
    # Detection of ArUco markers
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # If markers are detected
    if ids is not None:
        for i, marker_id in enumerate(ids):
            if marker_id < 4:  # get only the code 0-1-2-3 which are the corners
                keypoints_coordinates.append(corners[i][0][0])  # Append the corners of the marker
        keypoints_coordinates = [keypoint.astype(int).tolist() for keypoint in keypoints_coordinates]

        # Verify if there are 4 coordinates
        if len(keypoints_coordinates) == 4:
            # reference point for the calculation of the distance
            reference_points = np.array([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]], dtype=np.float32)
            
            def distance(point1, point2):
                return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

            # Distance between points and corners
            distances = np.array([[distance(point, ref) for ref in reference_points] for point in keypoints_coordinates])

            # Find corner with min distance for each corner
            closest_references = np.argmin(distances, axis=1)

            # Re-organisation of the points and keep the closest one
            corner_coordinates = [keypoints_coordinates[i] for i in np.argsort(closest_references)]
        else:
            corner_coordinates = None
    else :
        corner_coordinates = None
    return corner_coordinates;

In [3]:
def crop_image(img, coordinates):
    """
    Crop the image between the 4 patterns in the corners
    """
    new_width = 605
    new_height = 570
    
    # Use the coordinates to obtain the transformation matrix
    original_points = np.array(coordinates, dtype=np.float32)
    destination_points = np.array([[0, 0], [new_width-1, 0], [new_width-1, new_height-1], [0, new_height-1]], dtype=np.float32)

    matrix = cv2.getPerspectiveTransform(original_points, destination_points)

    # Apply the perspective transformation to the original image
    img_crop = cv2.warpPerspective(img, matrix, (new_width, new_height))
    return img_crop

In [4]:
def visionning(img,gray,img_crop,img_obstacles):
    """
    This function can combine 4 images in one for visualisation on live
    """
    img1 = img
    img2 = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    img3 = cv2.cvtColor(cv2.resize(img_crop, (gray.shape[1], gray.shape[0])), cv2.COLOR_GRAY2BGR)
    img4 = cv2.cvtColor(cv2.resize(img_obstacles, (gray.shape[1], gray.shape[0])), cv2.COLOR_GRAY2BGR)

    combined_image1 = np.hstack((img1, img2))
    combined_image2 = np.hstack((img3, img4))
    combined_image = np.vstack((combined_image1, combined_image2))
    return combined_image

In [5]:
def get_position_orientation_thymio(img):
    """
    Get the position and the orientation of the thymio
    """
    
    # Initialization of the ArUco dictionary
    aruco_dict = aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
    parameters = aruco.DetectorParameters()
    
    keypoints_coordinates = []
    
    # Detection of ArUco markers
    corners, ids, rejectedImgPoints = aruco.detectMarkers(img, aruco_dict, parameters=parameters)

    # If markers are detected
    if ids is not None:
        for i, marker_id in enumerate(ids):
            if marker_id == 4:  # get only the 4th which is the thymio
                
                # Position
                position = np.mean(corners[i][0], axis=0)
                #print("Position - \n", position)
                
                # Orientation of the robot
                vector = corners[i][0][0] - corners[i][0][3]
                x_axis_vector = np.array([1, 0])  # Vector along the x-axis
                angle_radians = np.arctan2(np.linalg.det([vector, x_axis_vector]), np.dot(vector, x_axis_vector))
                orientation = -np.degrees(angle_radians)
                #print("Orientation - \n", orientation)
                
                keypoints_coordinates.append(corners[i][0][0])  # Append the corners of the marker
        keypoints_coordinates = [keypoint.astype(int).tolist() for keypoint in keypoints_coordinates]
    # Placeholder for position and orientation (modify accordingly)
    raw_camera_measurement = [int(position[0]), int(position[1]), round(orientation,2)]

    #return position,orientation
    return raw_camera_measurement

In [6]:
def get_obstacles_coordinates(img,pos_goal):
    """
    Get the corner of the obstacles
        - mask the pattern on the Thymio
        - mask the corner's patterns
        - mask the goal
        - find contours and keep the corners
        - circle the edges on the img
    """
    
    # Initialization of the ArUco dictionary
    aruco_dict = aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
    parameters = aruco.DetectorParameters()
    
    keypoints_coordinates = []
    
    # Image without the marker on the Thymio
    corners, ids, rejectedImgPoints = aruco.detectMarkers(img, aruco_dict, parameters=parameters)
    for i, marker_id in enumerate(ids):
        #print(corners[i][0])
        x_coordinates = corners[i][0][:, 0]
        y_coordinates = corners[i][0][:, 1]
        min_x, max_x = np.min(x_coordinates)-5, np.max(x_coordinates)+5
        min_y, max_y = np.min(y_coordinates)-5, np.max(y_coordinates)+5
        
        # Create a white square between specified coordinates
        mask = np.zeros_like(img, dtype=np.uint8)
        mask[int(min_y):int(max_y), int(min_x):int(max_x)] = 255
        img_obstacles = cv2.bitwise_or(img, mask)
        
    size_marker = 40
    (h, w) = img_obstacles.shape
    #Images without the markers on the corner and the goal
    mask = np.zeros_like(img_obstacles, dtype=np.uint8)
    mask[0:size_marker, 0:size_marker] = 255
    mask[0:size_marker, w-size_marker:w] = 255
    mask[h-size_marker:h, 0:size_marker] = 255
    mask[h-size_marker:h, w-size_marker:w] = 255
    x1=int(pos_goal[0]-size_marker/2)
    x2=int(pos_goal[0]+size_marker/2)
    y1=int(pos_goal[1]-size_marker/2)
    y2=int(pos_goal[1]+size_marker/2)
    mask[y1:y2, x1:x2] = 255
    img_obstacles = cv2.bitwise_or(img_obstacles, mask)
    
    #reverse the image for the cv2.Canny()
    img_obstacles = cv2.bitwise_not(img_obstacles)
    pos_obstacle = []
    # Apply blur to reduce noise
    blurred = cv2.GaussianBlur(img_obstacles, (9, 9), 0)
    # Apply edge detection
    edges = cv2.Canny(blurred, 150, 200)

    # Find contours in the image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop through the contours
    for contour in contours:
        # Approximate the contour with a polygon
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Merge points that are too close        
        def distance(p1, p2):
            return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

        new_approx = []
        merged_points = set()

        for i in range(len(approx)):
            if i not in merged_points:
                current_point = approx[i][0]

                # Find points that are too close
                close_points = [j for j in range(i + 1, len(approx)) if distance(current_point, approx[j][0]) < 10]

                # Calculate the average of close points and update the list
                if close_points:
                    merged_points.update(close_points)
                    close_points.append(i)
                    average_point = np.mean(np.array([approx[j][0] for j in close_points]), axis=0)
                    new_approx.append([average_point.astype(int)])
                else:
                    new_approx.append([current_point])
        if len(new_approx)>2 and len(new_approx)<7:
            # Convert the resulting list to a NumPy array
            new_approx = np.array(new_approx)
            pos = [list(point[0]) for point in new_approx]
            #pos = [tuple(point[0]) for point in new_approx]
            pos_obstacle.append(pos)
            
            for point in new_approx:
                cv2.circle(img_obstacles, tuple(point[0]), 5, (0, 255, 0), -1)
    #print("pos_obstacle :", pos_obstacle)
    return img_obstacles,pos_obstacle

In [7]:
def find_goal(gray_image):
    """
    Find the coordinate of the goal
    """
    # Setup SimpleBlobDetector parameters.
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    params.minThreshold = 10
    params.maxThreshold = 200

    # Filter by Area.
    params.filterByArea = True
    params.minArea = 500
    params.maxArea = 2000

    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.9

    # Filter by Convexity
    params.filterByConvexity = True
    params.minConvexity = 0.9

    # Filter by Inertia3
    params.filterByInertia = True
    params.minInertiaRatio = 0.7

    # Create a blob detector with the specified parameters
    detector = cv2.SimpleBlobDetector_create(params)

    # Detect blobs.
    keypoints = detector.detect(gray_image)

    # Draw detected blobs as red circles.
    im_with_keypoints = cv2.drawKeypoints(gray_image, keypoints, np.array([]), (255, 0, 0),
                                          cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    pos_goal = []
    # Retrieve the x, y coordinates of each keypoint
    for keypoint in keypoints:
        x, y = keypoint.pt
        #pos_goal.append((x,y))
        pos_goal.append(int(x))
        pos_goal.append(int(y))
    #print("position goal : ",pos_goal)
    return pos_goal

In [8]:
"""def main():
    cam_port = 1 #camera 1 in my computer
    cam = cv2.VideoCapture(cam_port)
    corner_coordinates = None
    first_time = True

    # Check if the camera opened successfully
    if not cam.isOpened():
        print("Error opening camera")
        return

    try:
        while True:
            if first_time: # wait for focus of the camera because firsts image are not good
                img = read_camera_image(cam)
                time.sleep(0.5)
                
            # Read the camera image
            img = read_camera_image(cam)

            # If the image is read successfully, display it
            if img is not None:
                # Conversion to grayscale
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                if first_time:
                    corner_coordinates = get_corner(gray) #get the position of the obstacles
                    if corner_coordinates is not None: #if we had get the coordinates
                        img_crop = crop_image(gray,corner_coordinates) #crop image
                        pos_goal = find_goal(img_crop)  #find goal
                        if pos_goal is not None:
                            img_obstacles,pos_obstacle = get_obstacles_coordinates(img_crop,pos_goal) #get coordinates of the obstacles
                            first_time = False
                
                # Display the image with the detected markers
                if corner_coordinates is not None:
                    img_crop = crop_image(gray,corner_coordinates) #crop image
                    raw_camera_measurement = get_position_orientation_thymio(img_crop) #get position of thymio
                    print(raw_camera_measurement)
                    
                    cv2.imshow('Vision', visionning(img,gray,img_crop,img_obstacles))
                    
                else:
                    cv2.imshow('Vision', img)

            # Break the loop if the 'Esc' key is pressed
            if cv2.waitKey(100) == 27:  # ASCII code for 'Esc' key
                cam.release()
                cv2.destroyAllWindows()
                break   
            
            

    finally:
        # Release the camera capture object and close all windows
        cam.release()
        cv2.destroyAllWindows()

        
if __name__ == "__main__":
    main()"""

'def main():\n    cam_port = 1 #camera 1 in my computer\n    cam = cv2.VideoCapture(cam_port)\n    corner_coordinates = None\n    first_time = True\n\n    # Check if the camera opened successfully\n    if not cam.isOpened():\n        print("Error opening camera")\n        return\n\n    try:\n        while True:\n            if first_time: # wait for focus of the camera because firsts image are not good\n                img = read_camera_image(cam)\n                time.sleep(0.5)\n                \n            # Read the camera image\n            img = read_camera_image(cam)\n\n            # If the image is read successfully, display it\n            if img is not None:\n                # Conversion to grayscale\n                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n                \n                if first_time:\n                    corner_coordinates = get_corner(gray) #get the position of the obstacles\n                    if corner_coordinates is not None: #if we had get th

In [9]:
def init():
    cam_port = 1 #camera 1 in my computer
    cam = cv2.VideoCapture(cam_port)
    first_time = True

    # Check if the camera opened successfully
    if not cam.isOpened():
        print("Error opening camera")
        return None

    try:
        while True:
            if first_time: # wait for focus of the camera because firsts image are not good
                img = read_camera_image(cam)
                time.sleep(0.5)
                first_time = False
                
            # Read the camera image
            img = read_camera_image(cam)

            # If the image is read successfully, display it
            if img is not None:
                # Conversion to grayscale
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                corner_coordinates = get_corner(gray) #get the position of the corners
                if corner_coordinates is not None: #if we had get the coordinates
                    img_crop = crop_image(gray,corner_coordinates) #crop image
                    pos_goal = find_goal(img_crop)  #find goal
                    if pos_goal is not None:
                        img_obstacles,pos_obstacle = get_obstacles_coordinates(img_crop,pos_goal) #get coordinates of the obstacles
                        pos_thymio = get_position_orientation_thymio(img_crop) #get position of thymio
                        first_time = False
                        print("pos_goal : ", pos_goal)
                        print("pos_obstacle : ",pos_obstacle)
                        print("pos_thymio : ",pos_thymio)
                        width=60
                        goal_pos = [[pos_goal[0]-20, pos_goal[1]-20], [pos_goal[0]-20, pos_goal[1]+20], [pos_goal[0]+20, pos_goal[1]+20], [pos_goal[0]+20, pos_goal[1]-20]]
                        start_pos = [[pos_thymio[0]-50, pos_thymio[1]-50], [pos_thymio[0]-50, pos_thymio[1]+50], [pos_thymio[0]+50, pos_thymio[1]+50], [pos_thymio[0]+50, pos_thymio[1]-50]]
                        
                        #width = 0.2
                        #obs_list = [[[300, 200], [400, 200], [400, 300], [300, 300]], [[100, 200], [200, 400], [100, 400]]]
                        #start_pos = [[100, 100], [150, 100], [150, 150], [100, 150]]
                        robot_dir0 = 0
                        #goal_pos = [[450, 450], [500, 450], [500, 500], [450, 500]]
                        findPath(pos_obstacle, start_pos, robot_dir0, goal_pos, width)
                        #findPath(pos_obstacle, start_pos, pos_thymio[2], pos_goal, width)
                        return None
                
    finally:
        # Release the camera capture object and close all windows
        cam.release()
        cv2.destroyAllWindows()
        return None

In [10]:
def main():
    init()
        
if __name__ == "__main__":
    main()

pos_goal :  [511, 153]
pos_obstacle :  [[[97, 374], [40, 431], [135, 495], [162, 464]], [[282, 360], [282, 481], [346, 481], [346, 450], [314, 447], [313, 361]], [[480, 313], [438, 317], [448, 482], [494, 478]], [[99, 183], [84, 189], [67, 185]], [[89, 163], [94, 171], [91, 181]], [[52, 159], [54, 178], [81, 153], [96, 163], [103, 179], [65, 151]], [[38, 63], [40, 100], [79, 98], [76, 61], [52, 60]], [[364, 39], [283, 48], [287, 115], [329, 152], [370, 111]]]
pos_thymio :  [59, 80, -1.91]
